In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [2]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100,100] # MW

loads = ['Consumidor 1']
Nc = len(loads)
demanda_energia = [130]

scen = ['Cenario_1', 'Cenario_2']
Ns = len(scen)
prob_cenario = [0.05,0.95] # probabilidade de ocorência dos cenários
dem_reserva_real = pd.DataFrame([[20,10]], columns = scen, index = loads) # Carga por cenários
dem_reserva_forecast = np.dot(dem_reserva_real, prob_cenario) # Forecast da Carga


C_ener = [10,30] # $/MWh
C_res = [0,25]  # $/MW
C_shed = [100]  # $/MWh por carga

# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.c_res = Param(model.ger, initialize={ger[i]: C_res[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_shed = Param(model.loads, initialize={loads[i]: C_shed[i] for i in range(Nc)})

#Cenários:
model.scen = Set(initialize=scen)
model.prob = Param(model.scen, initialize={scen[i]: prob_cenario[i] for i in range(Ns)})
dem_res_real = {(loads[i], scen[j]): dem_reserva_real.loc[loads[i], scen[j]] for i in range(Nc) for j in range(Ns)}
model.dem_res_real = Param(model.loads, model.scen, initialize=dem_res_real)
dem_res_forecast = Param(model.loads, initialize={loads[i]: dem_reserva_forecast[i] for i in range(Nc)})

# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals)
model.R = Var(model.ger, domain=NonNegativeReals)
model.r = Var(model.ger, model.scen, domain=NonNegativeReals)
model.L_shed = Var(model.loads, model.scen, domain = NonNegativeReals)

#Restrições
def day_ahead_balance(model): #Restrição do balanço de energia no mercado diário
    thermal_generation =  sum(model.P_ger[g] for g in model.ger)
    demand = sum(model.dem[l] for l in model.loads)
    return thermal_generation == demand
model.day_ahead_balance = Constraint(rule=day_ahead_balance)

def upper_bounds(model, ger): # restrição 3: limite de geração
    return model.P_ger[ger] + model.R[ger] <= model.pmax_ter[ger]
model.UpperBound = Constraint(model.ger, rule=upper_bounds)

def non_ant_res(model, ger, cen):
    return model.r[ger,cen] <= model.R[ger]
model.NonAntRes = Constraint(model.ger, model.scen, rule=non_ant_res)

##MCCormick:
confianca = 0.95
#Variáveis: r e L_shed
model.z = Var(model.scen, domain=Binary)
model.k_r = Var(model.ger, model.scen, domain = Reals)
model.k_Lshed = Var(model.loads, model.scen, domain = Reals)

r_min = [0]*len(ger)
model.r_min = Param(model.ger, initialize={ger[i]: r_min[i] for i in range(Ng)})

L_shed_min = [0]*len(loads)
model.L_shed_min = Param(model.loads, initialize={loads[i]: L_shed_min[i] for i in range(Nc)})



#Restrições mccormick para r:
def mccormick_1_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] >= model.r_min[variavel] * model.z[cenario]

def mccormick_2_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] >= model.R[variavel] * model.z[cenario] + model.r[variavel, cenario] - model.R[variavel]

def mccormick_3_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] <= model.R[variavel] * model.z[cenario]

def mccormick_4_r(model, variavel, cenario):
      return model.k_r[variavel, cenario] <= model.r[variavel, cenario] + model.r_min[variavel] * model.z[cenario] - model.r_min[variavel]

model.mccormick_1_r = Constraint(model.ger, model.scen, rule=mccormick_1_r)
model.mccormick_2_r = Constraint(model.ger, model.scen, rule=mccormick_2_r)
model.mccormick_3_r = Constraint(model.ger, model.scen, rule=mccormick_3_r)
model.mccormick_4_r = Constraint(model.ger, model.scen, rule=mccormick_4_r)

#Restrições mccormick para L_shed:
def mccormick_1_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] >= model.L_shed_min[variavel] * model.z[cenario]

def mccormick_2_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] >= model.dem_res_real[variavel, cenario] * model.z[cenario] + model.L_shed[variavel, cenario] - model.dem_res_real[variavel, cenario]


def mccormick_3_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] <= model.dem_res_real[variavel, cenario] * model.z[cenario]

def mccormick_4_Lshed(model, variavel, cenario):
      return model.k_Lshed[variavel, cenario] <= model.L_shed[variavel, cenario] + model.L_shed_min[variavel] * model.z[cenario] - model.L_shed_min[variavel]

model.mccormick_1_Lshed = Constraint(model.loads, model.scen, rule=mccormick_1_Lshed)
model.mccormick_2_Lshed = Constraint(model.loads, model.scen, rule=mccormick_2_Lshed)
model.mccormick_3_Lshed = Constraint(model.loads, model.scen, rule=mccormick_3_Lshed)
model.mccormick_4_Lshed = Constraint(model.loads, model.scen, rule=mccormick_4_Lshed)

#Restrição confiança
def restricao_adicional_mccormick(model):
    return sum(model.z[cen] * model.prob[cen] for cen in model.scen) >= confianca
model.restricao_adicional = Constraint(rule=restricao_adicional_mccormick)

def operation_balance(model, carga, cen): #Restrição do balanço
    thermal_reserve_ativation = sum(model.k_r[g, cen] for g in model.ger)
    shed_load = sum(model.k_Lshed[l, cen] for l in model.loads)
    return thermal_reserve_ativation + shed_load == model.dem_res_real[carga,cen]*model.z[cen]
model.operation_balance = Constraint(model.loads, model.scen, rule=operation_balance)

# Declarar Função Objetivo
def FOB(model):
    #Parte Determinística
    custo_energia = sum(model.c_ener[g]*model.P_ger[g] for g in model.ger)
    custo_reserva = sum(model.c_res[g]*model.R[g] for g in model.ger)
    FOB_det = custo_energia + custo_reserva

    #Parte Probabilística
    FOB_prob = 0
    for cen in model.scen:
        custo_reserva_ativada = sum(model.c_ener[g]*model.r[g,cen] for g in model.ger)
        custo_deslastre = sum(model.c_shed[l]*model.L_shed[l,cen] for l in model.loads)
        FOB_prob += model.prob[cen] * (custo_reserva_ativada + custo_deslastre)
    return FOB_det + FOB_prob

model.objective = Objective(rule=FOB, sense=minimize)
solver = SolverFactory('ipopt')
resultados = solver.solve(model, tee=False)


In [3]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Print das Variáveis
for i in model.ger:
    print(f'Produção do {i}:' ,value(model.P_ger[i]), '\n')
    print(f'Reserva do {i}:' ,value(model.R[i]), '\n')


for j in model.scen:
    for i in model.loads:
        print(f'Deslastre do {i} no {j}:', value(model.L_shed[i,j]), '\n')
    for i in model.ger:
        print(f'Reserva ativada do {i} no {j}:' ,value(model.r[i,j]), '\n')

#Variáveis de decisão Z:
for i in model.scen:
    print(f'Z_{i}:' ,value(model.z[i]), '\n')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 2194.9999787145725 

Produção do Produtor A: 90.00000100981073 

Reserva do Produtor A: 9.999999990063978 

Produção do Produtor B: 39.999998990189276 

Reserva do Produtor B: 0.0 

Deslastre do Consumidor 1 no Cenario_1: 0.0 

Reserva ativada do Produtor A no Cenario_1: 3.0015996645152975e-09 

Reserva ativada do Produtor B no Cenario_1: 0.0 

Deslastre do Consumidor 1 no Cenario_2: 0.0 

Reserva ativada do Produtor A no Cenario_2: 9.999999885187991 

Reserva ativada do Produtor B no Cenario_2: 0.0 

Z_Cenario_1: 1.5429883423262977e-11 

Z_Cenario_2: 0.9999999894988796 

